# データ
次の辞書ファイルを作成する

## 姓名辞書
- oss.json
オープンソースの辞書から作成した，漢字・アルファベットとカタカナのペアデータ
- seimei.json
ダウンロードしたWebデータ（Wikipedia）から作成した漢字・アルファベットとカタカナのペアのデータ

## 単漢字辞書
- tankanji.json
オープンソースの辞書から作成した，単漢字とカタカナのペアデータ

## 異体字辞書
- itaiji.json
オープンソースの辞書から作成した，異体字の漢字のペアデータ




# データを準備する

辞書用及び。AI用のデータを，オープンソースのデータからダウンロードし，整形する。
使用するデータソースは次のとおり      

## データソース一覧

### オープンソース辞書データ
- [canna](https://osdn.dl.osdn.net/canna/)
- [freewnn](https://osdn.dl.osdn.net/freewnn/)
- [ipadic](https://taku910.github.io/mecab/)
- [kakasi](http://kakasi.namazu.org/)
- [mj](https://moji.or.jp/)
- [mozc](https://github.com/google/mozc/)
- [neologd](https://github.com/neologd/mecab-ipadic-neologd/)
- [skk](https://github.com/skk-dev/dict/)

### Webデータ
- [wikipedia](https://dumps.wikimedia.org/)

## データ形式について
データは，単語辞書，単漢字辞書，異体字辞書をそれぞれのデータソースから作成する。
作成するデータフォーマットはJSON形式であり，項目については次のとおり。

### 姓名辞書
単語漢字をキーとし，その読みのカタカナをキーとするハッシュテーブルをバリューとして追加する。またカタカナにはそれぞれdicsというキーで，漢字とカタカナのペアが存在する，データソースの一覧を保持している。
```json
{ 
    "単語漢字１" :{
        "カタカナ１":{
            "dics":[
                "freewnn",
                "canna"
            ]
        },
        "カタカナ２":{
            "dics":{
                "canna"
            }
        }
    },
    "単語漢字２":{
        ..
    }
..

```

### 単漢字辞書
単漢字をキーとし，その読みのカタカナをキーとするハッシュテーブルをバリューとして追加する。またカタカナにはそれぞれdicsというキーで，漢字とカタカナのペアが存在する，データソースの一覧を保持している。
```json
{ 
    "単漢字１" :{
        "カタカナ１":{
            "dics":[
                "freewnn",
                "canna"
            ]
        },
        "カタカナ２":{
            "dics":{
                "canna"
            }
        }
    },
    "単漢字２":{
        ..
    }
..

```


### 異体字辞書
単漢字をキーとし，その異体字である単漢字をキーとするハッシュテーブルをバリューとして追加する。また異体字である単漢字にはそれぞれdicsというキーで，漢字とカタカナのペアが存在する，データソースの一覧を保持している。

```json
{ 
    "単漢字１" :{
        "異体字１":{
            "dics":[
                "kakasi",
                "mozc"
            ]
        },
        "異体字２":{
            "dics":{
                "skk"
            }
        }
    },
    "単漢字２":{
        ..
    }
..

```

## 事前準備
pythonで必要なライブラリ群をインストールしておく

In [1]:
# warning off
import warnings
warnings.filterwarnings('ignore')

In [2]:
!pip install -r ../requirements.txt;




[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [3]:
!pip install wget


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


# データをダウンロードし，漢字とカタカナを抜き出す
あらかじめ，nkfコマンドをインストールしておく必要がある。

## canna
cannnaからは，単語辞書と単漢字辞書を作成する。単漢字辞書は単語辞書から一文字の漢字を抜き出したもの。


In [4]:
%cd canna
#!wget -nc -q "https://osdn.dl.osdn.net/canna/9565/Canna37p3.tar.bz2" --no-check-certificate;
#!bunzip2 Canna37p3.tar.bz2;
#!tar xfp Canna37p3.tar;
#!bzip2 Canna37p3.tar;
!rm -rf v3.7p3.tar.gz
!wget "https://github.com/netsphere-labs/canna/archive/refs/tags/v3.7p3.tar.gz"


/home/analysis01/src/kanjikana-model/dict/canna
--2025-03-18 15:38:44--  https://github.com/netsphere-labs/canna/archive/refs/tags/v3.7p3.tar.gz
github.com (github.com) をDNSに問いあわせています... 20.27.177.113
github.com (github.com)|20.27.177.113|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 302 Found
場所: https://codeload.github.com/netsphere-labs/canna/tar.gz/refs/tags/v3.7p3 [続く]
--2025-03-18 15:38:45--  https://codeload.github.com/netsphere-labs/canna/tar.gz/refs/tags/v3.7p3
codeload.github.com (codeload.github.com) をDNSに問いあわせています... 20.27.177.114
codeload.github.com (codeload.github.com)|20.27.177.114|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 特定できません [application/x-gzip]
`v3.7p3.tar.gz' に保存中

v3.7p3.tar.gz           [ <=>                ]   1.43M  7.86MB/s 時間 0.2s       

2025-03-18 15:38:45 (7.86 MB/s) - `v3.7p3.tar.gz' へ保存終了 [1497279]



In [5]:
# 単語辞書を作成する，漢字とカタカナのペアのCSV形式
!python extract_canna.py --indir Canna37p3/dic/ideo/pubdic --outfile extract.txt
!python words.py --indir "Canna37p3/dic/ideo/words" --outfile words.txt
!cat extract.txt words.txt > dict.txt


{
  "indir": "Canna37p3/dic/ideo/pubdic",
  "outfile": "extract.txt"
}
{
  "indir": "Canna37p3/dic/ideo/words",
  "outfile": "words.txt"
}


In [6]:
# 単漢字辞書を作成する。単語辞書から一文字の漢字を抜き出す
!python ../tankan.py --infile dict.txt --outfile tankanji.txt


{
  "infile": "dict.txt",
  "outfile": "tankanji.txt"
}


In [7]:
# cannaのダウンロードデータを削除する
!rm -rf Canna37p3
%cd ..


/home/analysis01/src/kanjikana-model/dict


## freewnn
freewnnからは，単語辞書と単漢字辞書を作成する。単漢字辞書は単語辞書から一文字の漢字を抜き出したもの。


In [8]:
%cd freewnn
#!wget -nc -q https://osdn.dl.osdn.net/freewnn/63271/FreeWnn-1.1.1-a023.tar.gz --no-check-certificate
#!tar zxfp FreeWnn-1.1.1-a023.tar.gz


!wget https://github.com/MItsutoshiNAKANO/FreeWnn/archive/refs/heads/master.zip


/home/analysis01/src/kanjikana-model/dict/freewnn
--2025-03-18 15:38:47--  https://github.com/MItsutoshiNAKANO/FreeWnn/archive/refs/heads/master.zip
github.com (github.com) をDNSに問いあわせています... 20.27.177.113
github.com (github.com)|20.27.177.113|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 302 Found
場所: https://codeload.github.com/MItsutoshiNAKANO/FreeWnn/zip/refs/heads/master [続く]
--2025-03-18 15:38:47--  https://codeload.github.com/MItsutoshiNAKANO/FreeWnn/zip/refs/heads/master
codeload.github.com (codeload.github.com) をDNSに問いあわせています... 20.27.177.114
codeload.github.com (codeload.github.com)|20.27.177.114|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 特定できません [application/zip]
`master.zip' に保存中

master.zip              [   <=>              ]   4.40M  8.66MB/s 時間 0.5s       

2025-03-18 15:38:48 (8.66 MB/s) - `master.zip' へ保存終了 [4615276]



In [9]:
# 単語辞書を作成する，漢字とカタカナのペアのCSV形式
!python extract_freewnn.py --infile FreeWnn-1.1.1-a023/PubdicPlus/pubdic.p --outfile dict.txt
!rm -rf FreeWnn-1.1.1-a023


{
  "infile": "FreeWnn-1.1.1-a023/PubdicPlus/pubdic.p",
  "outfile": "dict.txt"
}
Traceback (most recent call last):
  File "/home/analysis01/src/kanjikana-model/dict/freewnn/extract_freewnn.py", line 42, in <module>
    main()
  File "/home/analysis01/src/kanjikana-model/dict/freewnn/extract_freewnn.py", line 39, in main
    run(args)
  File "/home/analysis01/src/kanjikana-model/dict/freewnn/extract_freewnn.py", line 23, in run
    with open(args.infile,'r',encoding='euc-jp') as f:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'FreeWnn-1.1.1-a023/PubdicPlus/pubdic.p'


In [10]:
# 単漢字辞書を作成する。単語辞書から一文字の漢字を抜き出す
!python ../tankan.py --infile dict.txt --outfile tankanji.txt


{
  "infile": "dict.txt",
  "outfile": "tankanji.txt"
}
Traceback (most recent call last):
  File "/home/analysis01/src/kanjikana-model/dict/freewnn/../tankan.py", line 44, in <module>
    main()
  File "/home/analysis01/src/kanjikana-model/dict/freewnn/../tankan.py", line 41, in main
    run(args)
  File "/home/analysis01/src/kanjikana-model/dict/freewnn/../tankan.py", line 26, in run
    with open(args.infile,'r',encoding='utf-8') as f:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'dict.txt'


In [11]:

# freewnnのダウンロードデータを削除する
!rm -rf FreeWnn-1.1.1-a023
%cd ..


/home/analysis01/src/kanjikana-model/dict



## ipadic
ipadicからは，単語辞書と単漢字辞書を作成する。単漢字辞書は単語辞書から一文字の漢字を抜き出したもの。

In [1]:
%cd ipadic
!rm -rf *gz
!wget -nc -q --no-check-certificate  "https://github.com/MItsutoshiNAKANO/FreeWnn/archive/refs/heads/master.zip" -O mecab-ipadic-2.7.0-20070801.tar.gz
!tar zxfp mecab-ipadic-2.7.0-20070801.tar.gz




/home/analysis01/src/kanjikana-model/dict/ipadic


/home/analysis01/src/kanjikana-model/venv/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]



gzip: stdin: unexpected end of file
tar: Child returned status 1
tar: Error is not recoverable: exiting now


In [ ]:
# 単語辞書を作成する，漢字とカタカナのペアのCSV形式
!python extract_ipadic.py --indir="mecab-ipadic-2.7.0-20070801/" --outfile="dict.txt"



In [ ]:
# 単漢字辞書を作成する。単語辞書から一文字の漢字を抜き出す
!python ../tankan.py --infile dict.txt --outfile tankanji.txt



In [ ]:
# ダウンロードデータを削除する
!rm -rf mecab-ipadic-2.7.0-20070801
%cd ..


## kakasi
kakasiからは，単語辞書と単漢字辞書，異体字辞書を作成する。単漢字辞書は単語辞書から一文字の漢字を抜き出したもの。

In [ ]:
%cd kakasi
!wget -nc -q http://kakasi.namazu.org/stable/kakasi-2.3.6.tar.gz --no-check-certificate
!tar zxfp kakasi-2.3.6.tar.gz


In [ ]:
# 単語辞書を作成する，漢字とカタカナのペアのCSV形式
!python dict_kakasi.py --infile="kakasi-2.3.6/kakasidict" --outfile="dict.txt"



In [ ]:
# 単漢字辞書を作成する。単語辞書から一文字の漢字を抜き出す
!python ../tankan.py --infile dict.txt --outfile tankanji.txt

In [ ]:
# 異体字辞書を作成する
!python  itaiji_kakasi.py --infile="kakasi-2.3.6/itaijidict" --outfile="itaiji.txt"

In [ ]:
# ダウンロードデータを削除する
!rm -rf kakasi-2.3.6
%cd ..

## mj
mjからは単漢字辞書を作成する。データはExcel形式で提供されているので，Excel形式をCSV形式へ変換する[xlsx2csv](https://github.com/dilshod/xlsx2csv)を利用する。

In [ ]:
%cd mj
!wget -nc -q https://moji.or.jp/wp-content/uploads/2024/01/mji.00602.xlsx


In [ ]:
# excel形式からcsvへ変換する
!xlsx2csv mji.00602.xlsx mj.csv

In [ ]:
# 単漢字辞書を作成する。
!python extract_xlsx.py --infile mj.csv --outfile tankanji.txt

In [ ]:
# ダウンロードしたファイルと不要ファイルを削除する
!rm -rf *.csv
%cd ..

## mozc
mozcからは，単語辞書と単漢字辞書，異体字辞書を作成する。単漢字辞書は単語辞書から一文字の漢字を抜き出したもの。



In [ ]:
%cd mozc
!wget -nc -q "https://github.com/google/mozc/archive/refs/tags/2.29.5268.102.tar.gz" --no-check-certificate
!tar xfpz 2.29.5268.102.tar.gz

In [ ]:
# 単語辞書を作成する
!python extract_mozc.py --indir="mozc-2.29.5268.102/src/data/dictionary_oss/" --outfile="dict1.txt"
!python extract_oss.py --indir="mozc-2.29.5268.102/src/data/oss/" --outfile="dict2.txt"
!cat dict1.txt dict2.txt > dict.txt
!rm -rf dict1.txt dict2.txt


In [ ]:
# 単漢字辞書を作成する
!python extract_single.py --indir="mozc-2.29.5268.102/src/data/single_kanji/" --outfile="tankanji.txt"


In [ ]:
# 異体字辞書を作成する 
!python extract_variant.py --indir="mozc-2.29.5268.102/src/data/single_kanji/" --outfile="itaiji.txt"

In [ ]:
# 不要ファイルを削除する
!rm -rf mozc-2.29.5268.102
%cd ..


## neologd
ipadicからは，単語辞書と単漢字辞書を作成する。単漢字辞書は単語辞書から一文字の漢字を抜き出したもの。


In [ ]:
%cd neologd
!wget -nc -q https://github.com/neologd/mecab-ipadic-neologd/archive/refs/tags/v0.0.7.tar.gz --no-check-certificate
!tar zxf v0.0.7.tar.gz
!xz --decompress  mecab-ipadic-neologd-0.0.7/seed/*xz

In [ ]:
# 単語辞書を作成する，漢字とカタカナのペアのCSV形式
!python extract_neologd.py --indir="mecab-ipadic-neologd-0.0.7/seed" --outfile="dict.txt"



In [ ]:
# 単漢字辞書を作成する。単語辞書から一文字の漢字を抜き出す
!python ../tankan.py --infile dict.txt --outfile tankanji.txt




In [ ]:
# 不要ファイルを削除する

!rm -rf mecab-ipadic-neologd-0.0.7
%cd ..

## skk
単語辞書と異体字辞書，単漢字辞書を作成する。単漢字辞書は単語辞書から一文字の漢字を抜き出したもの。


In [ ]:
%cd skk
!wget -nc -q "https://github.com/skk-dev/dict/raw/master/SKK-JISYO.jinmei" --no-check-certificate
!wget -nc -q "https://github.com/skk-dev/dict/raw/master/SKK-JISYO.fullname" --no-check-certificate
!wget -nc -q https://github.com/skk-dev/dict/raw/master/SKK-JISYO.itaiji --no-check-certificate

In [ ]:
# 姓名辞書を作成する
!python jinmei.py --infile SKK-JISYO.jinmei --outfile skk_jinmei.txt
!python fullname.py --infile SKK-JISYO.fullname --outfile skk_fullname.txt
!cat skk_jinmei.txt skk_fullname.txt > dict.txt


In [ ]:
# 単漢字辞書を作成する
!python ../tankan.py --infile dict.txt --outfile tankanji.txt



In [ ]:
# 異体字辞書を作成する
!python itaiji_skk.py --infile SKK-JISYO.itaiji  --outfile itaiji.txt


In [ ]:
# 不要ファイルを削除する

%cd ..

## wikipedia
wikipediaからは，概要部分を抜き出して，文頭に　「漢字（読み）・・」　と記載されている場合に，漢字と読み仮名のペアを抜き出す。多少間違えている可能性があるので注意。


Wikipediaの構造解析から，Abstract部分に着目する

'''アルマ・マリア・マーラー＝ヴェルフェル'''（'''Alma Maria Mahler-Werfel''', [[1879年]][[8月31日]] – [[1964年]][[12月11日]]）は、[[オーストリア]]の作曲家[[グス
タフ・マーラー]]の妻。結婚前の姓はシンドラー（Schindler）。自身も作曲を行い、16>の歌曲が今日に残されている。華麗な男性遍歴で知られる。 


こんな形で入っている行をまずは抜き出す。     
- 見出しの最初に，'''人名'''
- ()の中に'''読み'''が入っている
- ()の中に[[yyyy年]]
- ()の中に[[mm月dd日]]


In [ ]:
%cd wikipedia
!wget -nc -q https://dumps.wikimedia.org/jawiki/latest/jawiki-latest-pages-articles.xml.bz2 --no-check-certificate



In [ ]:
# Wikipediaから概要だけを抜き出す
!python extract_wiki.py --infile="jawiki-latest-pages-articles.xml.bz2" --outfile="abstract.txt"



In [ ]:
# 漢字とカナのペアとなるものと推測されるものを抜き出す。
!python extract_name.py --outfile="abstract_name.txt" --infile="abstract.txt";


In [ ]:
# extract_name.pyによってWikipediaのダンプファイルから，概要部分が抜き出されたものに対して，漢字・アルファベットとカタカナのペアのうち，正しいと推測されるものを選択する。
!python select_name.py --infile abstract_name.txt --outfile wikiname.txt --ngfile ng.txt;


In [ ]:
# select_name.pyによってWikipediaのダンプファイルから作成した漢字・アルファベットとカタカナのペアを単語単位に分割する
!python dict_name.py --infile wikiname.txt --outfile dict.txt

In [ ]:
# 不要ファイル削除
%cd ..


# ダウンロードしたデータから辞書を作成する

## オープンソース辞書データ
オープンソースの辞書から抜き出したデータで，oss.json,tankanji.json, itaiji.jsonを作成する


In [ ]:
%cd dict

In [ ]:
# oss.json
!rm -rf oss.json
!python merge.py --infile canna/dict.txt --jsonfile oss.json --dicname canna;
!python merge.py --infile freewnn/dict.txt --jsonfile oss.json --dicname freewnn;
!python merge.py --infile ipadic/dict.txt --jsonfile oss.json --dicname ipadic;
!python merge.py --infile kakasi/dict.txt --jsonfile oss.json --dicname kakasi;
!python merge.py --infile mozc/dict.txt --jsonfile oss.json --dicname mozc;
!python merge.py --infile neologd/dict.txt --jsonfile oss.json --dicname neologd;
!python merge.py --infile skk/dict.txt --jsonfile oss.json --dicname skk;

In [ ]:
# tankanji.json
!rm -rf tankanji.json

!python merge.py --infile canna/tankanji.txt --jsonfile tankanji.json --dicname canna;
!python merge.py --infile freewnn/tankanji.txt --jsonfile tankanji.json --dicname freewnn;
!python merge.py --infile ipadic/tankanji.txt --jsonfile tankanji.json --dicname ipadic;
!python merge.py --infile kakasi/tankanji.txt --jsonfile tankanji.json --dicname kakasi;
!python merge.py --infile mozc/tankanji.txt --jsonfile tankanji.json --dicname mozc;
!python merge.py --infile neologd/tankanji.txt --jsonfile tankanji.json --dicname neologd;
!python merge.py --infile skk/tankanji.txt --jsonfile tankanji.json --dicname skk;
!python merge.py --infile mj/tankanji.txt --jsonfile tankanji.json --dicname mj;

In [ ]:
# itaiji.json
!rm -rf itaiji.json
!python merge.py --infile kakasi/itaiji.txt --jsonfile itaiji.json --dicname kakasi
!python merge.py --infile mozc/itaiji.txt --jsonfile itaiji.json --dicname mozc
!python merge.py --infile skk/itaiji.txt --jsonfile itaiji.json --dicname skk


## Webデータ
wikipediaから抜き出したデータから，姓名の漢字とカナのペアで，seimei.jsonを作成する


In [ ]:
%cd wikipedia
# オープンソースの辞書のデータに入っていない漢字とカナのペアを抜き出す
!python check_dict.py --infile dict.txt --outfile dict_add.txt --oss ../oss.json;
%cd ..

In [ ]:
!rm -rf seimei.json
!python merge.py --infile wikipedia/dict.txt --jsonfile seimei.json --dicname wikipedia


## その他
その他，氏名漢字カナ突合モデルに必要な空ファイルを作成する


In [ ]:
import os

with open("crawl.json",'w',encoding='utf-8') as f:
    f.write("{}")

with open("statistics.json",'w',encoding='utf-8') as f:
    f.write("{}")


In [ ]:
!date